In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import csv

In [2]:
datafile_name = "Datasets\Life_Expectancy\Life Expectancy Data.csv"

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# Import data from the csv file

df = pd.read_csv(datafile_name)

In [14]:
# Visual the data
# Life expectancy - targets
Addition_Information_Title = ["Life expectancy ", "Status"]
Addition_Information = []

df = df.drop(columns=Addition_Information_Title, axis=1)
X = df
y = df["Life expectancy "]

X.head()

KeyError: "['Life expectancy ', 'Status'] not found in axis"

In [6]:
# Split the dataset


X_train , X_test_train,y_train, y_test_train = train_test_split(X,y,test_size=0.2)
X_validation, X_test_val, y_validation, y_test_val = train_test_split(X_train, y_train, test_size = 0.2)

In [7]:
print(X_test_train.shape)

(588, 20)


SyntaxError: incomplete input (214183014.py, line 3)